In [1]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from dataset.data_loader_RGB import *
from utils.pytorch_ssim import *

from models.FSRCNN_coord_model import Net

torch.manual_seed(1)
device = torch.device("cuda")

In [2]:
# FSRCNN parameters

batch_size = 24
epochs = 50
lr = 0.001
threads = 4
upscale_factor = 4

In [3]:
#img_path_low = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-56/train'
#img_path_ref = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/train'

img_path_low = '/media/angelo/DATEN/Datasets/CelebA/LR_56/train/'
img_path_ref = '/media/angelo/DATEN/Datasets/CelebA/HR/train/'

#img_path_low = '/home/jupyter/dataset/LR_56/train/'
#img_path_ref = '/home/jupyter/dataset/HR/train/'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [4]:
print('===> Building model')
model = Net().to(device)
model.weight_init(mean=0.0, std=0.2)

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.5, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.2)

===> Building model


In [5]:
out_path = 'results/'
out_model_path = 'models/'

if not os.path.exists(out_path):
    os.makedirs(out_path)    

if not os.path.exists(out_model_path):
    os.makedirs(out_model_path)   
    
results = {'avg_loss': [], 'psnr': [], 'ssim': []}

In [6]:
def train(epoch):
    epoch_loss = 0
    model.train()
    for iteration, batch in enumerate(training_data_loader, 1):
        input_, target = batch[0].to(device), batch[1].to(device)
        
        optimizer.zero_grad()
        upsampled_img = model(input_)
        loss = criterion(upsampled_img, target)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(training_data_loader), loss.item()))
    
    scheduler.step() # Decrease learning rate after 15 epochs to 10% of its value
    
    psnr_epoch = 10*log10(1/(epoch_loss / len(training_data_loader)))
    ssim_epoch = ssim(upsampled_img, target).item()
    avg_loss_batch = epoch_loss/len(training_data_loader)
    
    results['psnr'].append(psnr_epoch)
    results['ssim'].append(ssim_epoch)
    results['avg_loss'].append(avg_loss_batch)
    
    print("===> Epoch {} Complete: Avg. Loss: {:.4f} / PSNR: {:.4f} / SSIM {:.4f}".format(epoch, 
                                                                                          avg_loss_batch, 
                                                                                          psnr_epoch,
                                                                                          ssim_epoch))
    if epoch % (epochs // 10) == 0:
    
        data_frame = pd.DataFrame(
                data={'Avg. Loss': results['avg_loss'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
                index=range(1, epoch + 1))

        data_frame.to_csv(out_path + 'FSRCNN_coord_x' + str(upscale_factor) + '_train_results.csv', index_label='Epoch')
        
        checkpoint(epoch)
    
def checkpoint(epoch):
    path = out_model_path + "FSRCNN_coord_x{}_epoch_{}.pth".format(upscale_factor, epoch)
    torch.save(model, path)
    print("Checkpoint saved to {}".format(path))

In [ ]:
#epochs=10
#optimizer.param_groups[0]['lr'] = 0.001

for epoch in range(1, epochs+1):
    train(epoch)

===> Epoch[1](1/750): Loss: 0.3106
===> Epoch[1](2/750): Loss: 0.2545
===> Epoch[1](3/750): Loss: 0.2668
===> Epoch[1](4/750): Loss: 0.2270
===> Epoch[1](5/750): Loss: 0.2074
===> Epoch[1](6/750): Loss: 0.2002
===> Epoch[1](7/750): Loss: 0.1760
===> Epoch[1](8/750): Loss: 0.1647
===> Epoch[1](9/750): Loss: 0.1481
===> Epoch[1](10/750): Loss: 0.1338
===> Epoch[1](11/750): Loss: 0.1223
===> Epoch[1](12/750): Loss: 0.0936
===> Epoch[1](13/750): Loss: 0.0954
===> Epoch[1](14/750): Loss: 0.0784
===> Epoch[1](15/750): Loss: 0.0729
===> Epoch[1](16/750): Loss: 0.0702
===> Epoch[1](17/750): Loss: 0.0656
===> Epoch[1](18/750): Loss: 0.0583
===> Epoch[1](19/750): Loss: 0.0536
===> Epoch[1](20/750): Loss: 0.0541
===> Epoch[1](21/750): Loss: 0.0448
===> Epoch[1](22/750): Loss: 0.0505
===> Epoch[1](23/750): Loss: 0.0412
===> Epoch[1](24/750): Loss: 0.0433
===> Epoch[1](25/750): Loss: 0.0362
===> Epoch[1](26/750): Loss: 0.0436
===> Epoch[1](27/750): Loss: 0.0359
===> Epoch[1](28/750): Loss: 0.0361
=

===> Epoch[1](226/750): Loss: 0.0062
===> Epoch[1](227/750): Loss: 0.0057
===> Epoch[1](228/750): Loss: 0.0057
===> Epoch[1](229/750): Loss: 0.0060
===> Epoch[1](230/750): Loss: 0.0065
===> Epoch[1](231/750): Loss: 0.0056
===> Epoch[1](232/750): Loss: 0.0057
===> Epoch[1](233/750): Loss: 0.0060
===> Epoch[1](234/750): Loss: 0.0052
===> Epoch[1](235/750): Loss: 0.0063
===> Epoch[1](236/750): Loss: 0.0056
===> Epoch[1](237/750): Loss: 0.0048
===> Epoch[1](238/750): Loss: 0.0058
===> Epoch[1](239/750): Loss: 0.0057
===> Epoch[1](240/750): Loss: 0.0048
===> Epoch[1](241/750): Loss: 0.0059
===> Epoch[1](242/750): Loss: 0.0053
===> Epoch[1](243/750): Loss: 0.0061
===> Epoch[1](244/750): Loss: 0.0059
===> Epoch[1](245/750): Loss: 0.0056
===> Epoch[1](246/750): Loss: 0.0057
===> Epoch[1](247/750): Loss: 0.0059
===> Epoch[1](248/750): Loss: 0.0054
===> Epoch[1](249/750): Loss: 0.0053
===> Epoch[1](250/750): Loss: 0.0070
===> Epoch[1](251/750): Loss: 0.0049
===> Epoch[1](252/750): Loss: 0.0049
=

===> Epoch[1](449/750): Loss: 0.0036
===> Epoch[1](450/750): Loss: 0.0035
===> Epoch[1](451/750): Loss: 0.0031
===> Epoch[1](452/750): Loss: 0.0027
===> Epoch[1](453/750): Loss: 0.0036
===> Epoch[1](454/750): Loss: 0.0033
===> Epoch[1](455/750): Loss: 0.0034
===> Epoch[1](456/750): Loss: 0.0031
===> Epoch[1](457/750): Loss: 0.0031
===> Epoch[1](458/750): Loss: 0.0031
===> Epoch[1](459/750): Loss: 0.0029
===> Epoch[1](460/750): Loss: 0.0033
===> Epoch[1](461/750): Loss: 0.0037
===> Epoch[1](462/750): Loss: 0.0029
===> Epoch[1](463/750): Loss: 0.0029
===> Epoch[1](464/750): Loss: 0.0028
===> Epoch[1](465/750): Loss: 0.0027
===> Epoch[1](466/750): Loss: 0.0033
===> Epoch[1](467/750): Loss: 0.0032
===> Epoch[1](468/750): Loss: 0.0030
===> Epoch[1](469/750): Loss: 0.0031
===> Epoch[1](470/750): Loss: 0.0034
===> Epoch[1](471/750): Loss: 0.0032
===> Epoch[1](472/750): Loss: 0.0030
===> Epoch[1](473/750): Loss: 0.0030
===> Epoch[1](474/750): Loss: 0.0033
===> Epoch[1](475/750): Loss: 0.0030
=

===> Epoch[1](672/750): Loss: 0.0027
===> Epoch[1](673/750): Loss: 0.0022
===> Epoch[1](674/750): Loss: 0.0031
===> Epoch[1](675/750): Loss: 0.0023
===> Epoch[1](676/750): Loss: 0.0022
===> Epoch[1](677/750): Loss: 0.0025
===> Epoch[1](678/750): Loss: 0.0023
===> Epoch[1](679/750): Loss: 0.0024
===> Epoch[1](680/750): Loss: 0.0021
===> Epoch[1](681/750): Loss: 0.0027
===> Epoch[1](682/750): Loss: 0.0027
===> Epoch[1](683/750): Loss: 0.0026
===> Epoch[1](684/750): Loss: 0.0024
===> Epoch[1](685/750): Loss: 0.0029
===> Epoch[1](686/750): Loss: 0.0027
===> Epoch[1](687/750): Loss: 0.0027
===> Epoch[1](688/750): Loss: 0.0025
===> Epoch[1](689/750): Loss: 0.0024
===> Epoch[1](690/750): Loss: 0.0025
===> Epoch[1](691/750): Loss: 0.0029
===> Epoch[1](692/750): Loss: 0.0025
===> Epoch[1](693/750): Loss: 0.0027
===> Epoch[1](694/750): Loss: 0.0032
===> Epoch[1](695/750): Loss: 0.0026
===> Epoch[1](696/750): Loss: 0.0027
===> Epoch[1](697/750): Loss: 0.0025
===> Epoch[1](698/750): Loss: 0.0028
=

===> Epoch[2](145/750): Loss: 0.0026
===> Epoch[2](146/750): Loss: 0.0020
===> Epoch[2](147/750): Loss: 0.0020
===> Epoch[2](148/750): Loss: 0.0022
===> Epoch[2](149/750): Loss: 0.0024
===> Epoch[2](150/750): Loss: 0.0026
===> Epoch[2](151/750): Loss: 0.0020
===> Epoch[2](152/750): Loss: 0.0023
===> Epoch[2](153/750): Loss: 0.0022
===> Epoch[2](154/750): Loss: 0.0021
===> Epoch[2](155/750): Loss: 0.0020
===> Epoch[2](156/750): Loss: 0.0022
===> Epoch[2](157/750): Loss: 0.0023
===> Epoch[2](158/750): Loss: 0.0020
===> Epoch[2](159/750): Loss: 0.0024
===> Epoch[2](160/750): Loss: 0.0023
===> Epoch[2](161/750): Loss: 0.0023
===> Epoch[2](162/750): Loss: 0.0024
===> Epoch[2](163/750): Loss: 0.0027
===> Epoch[2](164/750): Loss: 0.0026
===> Epoch[2](165/750): Loss: 0.0022
===> Epoch[2](166/750): Loss: 0.0021
===> Epoch[2](167/750): Loss: 0.0019
===> Epoch[2](168/750): Loss: 0.0023
===> Epoch[2](169/750): Loss: 0.0022
===> Epoch[2](170/750): Loss: 0.0024
===> Epoch[2](171/750): Loss: 0.0023
=

...

===> Epoch[9](681/750): Loss: 0.0016
===> Epoch[9](682/750): Loss: 0.0017
===> Epoch[9](683/750): Loss: 0.0016
===> Epoch[9](684/750): Loss: 0.0015
===> Epoch[9](685/750): Loss: 0.0014
===> Epoch[9](686/750): Loss: 0.0019
===> Epoch[9](687/750): Loss: 0.0020
===> Epoch[9](688/750): Loss: 0.0016
===> Epoch[9](689/750): Loss: 0.0018
===> Epoch[9](690/750): Loss: 0.0017
===> Epoch[9](691/750): Loss: 0.0016
===> Epoch[9](692/750): Loss: 0.0020
===> Epoch[9](693/750): Loss: 0.0015
===> Epoch[9](694/750): Loss: 0.0015
===> Epoch[9](695/750): Loss: 0.0015
===> Epoch[9](696/750): Loss: 0.0016
===> Epoch[9](697/750): Loss: 0.0016
===> Epoch[9](698/750): Loss: 0.0018
===> Epoch[9](699/750): Loss: 0.0017
===> Epoch[9](700/750): Loss: 0.0019
===> Epoch[9](701/750): Loss: 0.0014
===> Epoch[9](702/750): Loss: 0.0015
===> Epoch[9](703/750): Loss: 0.0015
===> Epoch[9](704/750): Loss: 0.0017
===> Epoch[9](705/750): Loss: 0.0013
===> Epoch[9](706/750): Loss: 0.0016
===> Epoch[9](707/750): Loss: 0.0016
=

===> Epoch[10](151/750): Loss: 0.0016
===> Epoch[10](152/750): Loss: 0.0015
===> Epoch[10](153/750): Loss: 0.0019
===> Epoch[10](154/750): Loss: 0.0013
===> Epoch[10](155/750): Loss: 0.0019
===> Epoch[10](156/750): Loss: 0.0018
===> Epoch[10](157/750): Loss: 0.0014
===> Epoch[10](158/750): Loss: 0.0015
===> Epoch[10](159/750): Loss: 0.0017
===> Epoch[10](160/750): Loss: 0.0017
===> Epoch[10](161/750): Loss: 0.0016
===> Epoch[10](162/750): Loss: 0.0017
===> Epoch[10](163/750): Loss: 0.0014
===> Epoch[10](164/750): Loss: 0.0016
===> Epoch[10](165/750): Loss: 0.0017
===> Epoch[10](166/750): Loss: 0.0016
===> Epoch[10](167/750): Loss: 0.0014
===> Epoch[10](168/750): Loss: 0.0016
===> Epoch[10](169/750): Loss: 0.0019
===> Epoch[10](170/750): Loss: 0.0017
===> Epoch[10](171/750): Loss: 0.0018
===> Epoch[10](172/750): Loss: 0.0016
===> Epoch[10](173/750): Loss: 0.0016
===> Epoch[10](174/750): Loss: 0.0015
===> Epoch[10](175/750): Loss: 0.0015
===> Epoch[10](176/750): Loss: 0.0020
===> Epoch[1

===> Epoch[10](367/750): Loss: 0.0015
===> Epoch[10](368/750): Loss: 0.0016
===> Epoch[10](369/750): Loss: 0.0016
===> Epoch[10](370/750): Loss: 0.0015
===> Epoch[10](371/750): Loss: 0.0017
===> Epoch[10](372/750): Loss: 0.0016
===> Epoch[10](373/750): Loss: 0.0019
===> Epoch[10](374/750): Loss: 0.0017
===> Epoch[10](375/750): Loss: 0.0016
===> Epoch[10](376/750): Loss: 0.0020
===> Epoch[10](377/750): Loss: 0.0016
===> Epoch[10](378/750): Loss: 0.0015
===> Epoch[10](379/750): Loss: 0.0012
===> Epoch[10](380/750): Loss: 0.0018
===> Epoch[10](381/750): Loss: 0.0013
===> Epoch[10](382/750): Loss: 0.0016
===> Epoch[10](383/750): Loss: 0.0016
===> Epoch[10](384/750): Loss: 0.0015
===> Epoch[10](385/750): Loss: 0.0017
===> Epoch[10](386/750): Loss: 0.0016
===> Epoch[10](387/750): Loss: 0.0013
===> Epoch[10](388/750): Loss: 0.0017
===> Epoch[10](389/750): Loss: 0.0017
===> Epoch[10](390/750): Loss: 0.0014
===> Epoch[10](391/750): Loss: 0.0016
===> Epoch[10](392/750): Loss: 0.0015
===> Epoch[1

===> Epoch[10](583/750): Loss: 0.0016
===> Epoch[10](584/750): Loss: 0.0017
===> Epoch[10](585/750): Loss: 0.0013
===> Epoch[10](586/750): Loss: 0.0017
===> Epoch[10](587/750): Loss: 0.0015
===> Epoch[10](588/750): Loss: 0.0014
===> Epoch[10](589/750): Loss: 0.0016
===> Epoch[10](590/750): Loss: 0.0019
===> Epoch[10](591/750): Loss: 0.0017
===> Epoch[10](592/750): Loss: 0.0017
===> Epoch[10](593/750): Loss: 0.0015
===> Epoch[10](594/750): Loss: 0.0019
===> Epoch[10](595/750): Loss: 0.0016
===> Epoch[10](596/750): Loss: 0.0016
===> Epoch[10](597/750): Loss: 0.0015
===> Epoch[10](598/750): Loss: 0.0015
===> Epoch[10](599/750): Loss: 0.0014
===> Epoch[10](600/750): Loss: 0.0016
===> Epoch[10](601/750): Loss: 0.0015
===> Epoch[10](602/750): Loss: 0.0013
===> Epoch[10](603/750): Loss: 0.0016
===> Epoch[10](604/750): Loss: 0.0015
===> Epoch[10](605/750): Loss: 0.0017
===> Epoch[10](606/750): Loss: 0.0017
===> Epoch[10](607/750): Loss: 0.0015
===> Epoch[10](608/750): Loss: 0.0014
===> Epoch[1

===> Epoch[11](47/750): Loss: 0.0015
===> Epoch[11](48/750): Loss: 0.0015
===> Epoch[11](49/750): Loss: 0.0015
===> Epoch[11](50/750): Loss: 0.0016
===> Epoch[11](51/750): Loss: 0.0014
===> Epoch[11](52/750): Loss: 0.0016
===> Epoch[11](53/750): Loss: 0.0014
===> Epoch[11](54/750): Loss: 0.0014
===> Epoch[11](55/750): Loss: 0.0018
===> Epoch[11](56/750): Loss: 0.0013
===> Epoch[11](57/750): Loss: 0.0015
===> Epoch[11](58/750): Loss: 0.0018
===> Epoch[11](59/750): Loss: 0.0016
===> Epoch[11](60/750): Loss: 0.0017
===> Epoch[11](61/750): Loss: 0.0014
===> Epoch[11](62/750): Loss: 0.0017
===> Epoch[11](63/750): Loss: 0.0014
===> Epoch[11](64/750): Loss: 0.0017
===> Epoch[11](65/750): Loss: 0.0015
===> Epoch[11](66/750): Loss: 0.0015
===> Epoch[11](67/750): Loss: 0.0014
===> Epoch[11](68/750): Loss: 0.0018
===> Epoch[11](69/750): Loss: 0.0016
===> Epoch[11](70/750): Loss: 0.0016
===> Epoch[11](71/750): Loss: 0.0015
===> Epoch[11](72/750): Loss: 0.0017
===> Epoch[11](73/750): Loss: 0.0016
=